In [1]:
#!/usr/bin/env python
# coding: utf-8

import pandas as pd
import numpy as np
import os
import re
import json

#cwd = dir_path = os.path.dirname(os.path.realpath(__file__)) 
# bidsdir = os.path.join('%s/../bids'%(cwd))
cwd='/ZPOOL/data/projects/rf1-mbme-pilot/code'
bidsdir='/ZPOOL/data/projects/rf1-mbme-pilot/bids/'

print(bidsdir)

/ZPOOL/data/projects/rf1-mbme-pilot/bids/


In [2]:
event_files=[os.path.join(root,f) for root,dirs,files in os.walk(bidsdir) for f in files if f.endswith('events.tsv')]
#print(event_files)

data=[]
for file in event_files:
	sub='sub-'+re.search('func/sub-(.*)_task',file).group(1)
	acq=re.search('_acq-(.*)_events',file).group(1)
	tmp_df=pd.read_csv(file,sep='\t')
#	print(tmp_df)
	if tmp_df.shape[0]>0:
		#print(sub,acq)
		tmp_df['sub']=sub
		tmp_df['acq']=acq
		data.append(tmp_df)
events_df=pd.concat(data)

#print(events_df)

print("Unique subs:")
print(events_df['sub'].unique())


Unique subs:
['sub-10422' 'sub-10186' 'sub-10382' 'sub-10150' 'sub-10391' 'sub-10296'
 'sub-10590sp' 'sub-10741sp' 'sub-10043' 'sub-10723sp' 'sub-10198'
 'sub-10223' 'sub-10659sp' 'sub-10015' 'sub-10318' 'sub-10185'
 'sub-10803sp' 'sub-10603sp' 'sub-10320' 'sub-10054' 'sub-10008'
 'sub-10589sp' 'sub-10438' 'sub-10234' 'sub-10137' 'sub-10030'
 'sub-10608sp' 'sub-10130' 'sub-10078' 'sub-10108' 'sub-10024' 'sub-10059'
 'sub-10154' 'sub-10188' 'sub-10166' 'sub-10085' 'sub-10644sp' 'sub-10416'
 'sub-10026' 'sub-10606sp' 'sub-10014' 'sub-10069' 'sub-10035' 'sub-10203'
 'sub-10094' 'sub-10007' 'sub-10303' 'sub-10142' 'sub-12042' 'sub-10080'
 'sub-10777sp' 'sub-10221' 'sub-10691sp' 'sub-10017' 'sub-10363'
 'sub-10041' 'sub-10003' 'sub-10074' 'sub-10136' 'sub-10319' 'sub-10010'
 'sub-10321' 'sub-10690sp' 'sub-10125' 'sub-10028']


In [3]:
data=[]
for sub in events_df['sub'].unique():
    #print(sub)
    for acq in events_df['acq'].unique():
        
        # What is this fname file used for? Does not exist in file tree
        #fname='%s/../derivatives/fsl/mcflirt/%s/%s/_abs.rms'%(cwd,sub,acq)
        #print(fname)
        #if os.path.exists(fname):
        #absolute=np.loadtxt(fname)

            #if acq in ['mb1me4','mb3me4','mb6me4', 'mb3mb4fa50','mb3me3ip0','mb2me4','mb3me3']:
            #    f = open("%s/../derivatives/mriqc/%s/func/%s_task-sharedreward_acq-%s_echo-2_bold.json"%(cwd,sub,sub,acq))
            #else:
            #    f = open("%s/../derivatives/mriqc/%s/func/%s_task-sharedreward_acq-%s_bold.json"%(cwd,sub,sub,acq))
            #print(f)
            
            if acq in ['mb1me4','mb3me4','mb6me4', 'mb3mb4fa50','mb3me3ip0','mb2me4','mb3me3']:
                f = open("/ZPOOL/data/projects/rf1-mbme-pilot/derivatives/mriqc/%s/func/%s_task-sharedreward_acq-%s_echo-2_bold.json"%(sub,sub,acq))
            else:
                f = open("/ZPOOL/data/projects/rf1-mbme-pilot/derivatives/mriqc/%s/func/%s_task-sharedreward_acq-%s_bold.json"%(sub,sub,acq))
            print(f)
            
            # returns JSON object as a dictionary
            FD = json.load(f)
            f.close
            FD=FD['fd_mean']
            #print(FD)
            #FD=np.loadtxt('%s/../derivatives/fsl/mcflirt/%s/%s/_rel.rms'%(cwd,sub,acq))

            #row=[sub,acq,
            #    events_df[(events_df['sub']==sub)&(events_df['acq']==acq)]['trial_type'].str.count('miss_decision').sum(),
            #    absolute.max(),FD]
            row=[sub,acq,
                events_df[(events_df['sub']==sub)&(events_df['acq']==acq)]['trial_type'].str.count('miss_decision').sum(),
                FD]
            data.append(row)

exclusions_df=pd.DataFrame(data=data,columns=['sub','acq','TrialCount_misses','Max_Abs_motion','FD_mean'])
exclusions_df['FD_exclusion']=exclusions_df['FD_mean']>0.5
exclusions_df['ABS_exclusion']=exclusions_df['Max_Abs_motion']>1.35
exclusions_df['Beh_TrialExclusion']=exclusions_df['TrialCount_misses']>27
print(exclusions_df)

<_io.TextIOWrapper name='/ZPOOL/data/projects/rf1-mbme-pilot/derivatives/mriqc/sub-10422/func/sub-10422_task-sharedreward_acq-mb6me4_echo-2_bold.json' mode='r' encoding='UTF-8'>
<_io.TextIOWrapper name='/ZPOOL/data/projects/rf1-mbme-pilot/derivatives/mriqc/sub-10422/func/sub-10422_task-sharedreward_acq-mb1me1_bold.json' mode='r' encoding='UTF-8'>
<_io.TextIOWrapper name='/ZPOOL/data/projects/rf1-mbme-pilot/derivatives/mriqc/sub-10422/func/sub-10422_task-sharedreward_acq-mb3me4_echo-2_bold.json' mode='r' encoding='UTF-8'>
<_io.TextIOWrapper name='/ZPOOL/data/projects/rf1-mbme-pilot/derivatives/mriqc/sub-10422/func/sub-10422_task-sharedreward_acq-mb3me1_bold.json' mode='r' encoding='UTF-8'>
<_io.TextIOWrapper name='/ZPOOL/data/projects/rf1-mbme-pilot/derivatives/mriqc/sub-10422/func/sub-10422_task-sharedreward_acq-mb1me4_echo-2_bold.json' mode='r' encoding='UTF-8'>
<_io.TextIOWrapper name='/ZPOOL/data/projects/rf1-mbme-pilot/derivatives/mriqc/sub-10422/func/sub-10422_task-sharedreward_ac

FileNotFoundError: [Errno 2] No such file or directory: '/ZPOOL/data/projects/rf1-mbme-pilot/derivatives/mriqc/sub-10422/func/sub-10422_task-sharedreward_acq-mb3me3ip0_echo-2_bold.json'

In [ ]:
results=exclusions_df.groupby(by='sub').sum().reset_index().rename(columns={"TrialCount_misses": "TotalCount_misses"})
print(results)
results['Beh_TotalExclusion']=results['TotalCount_misses']>81
results=results[['sub','TotalCount_misses','Beh_TotalExclusion']]

In [ ]:
exclusions_df.merge(results,on='sub')
exclusions_df.to_csv('../derivatives/exclusions.csv', index=False)